# Pipeline 

Telegram bot (bot.py) \
Stocks API (stock.py) \
DataBase (database.py) \

Maybe to add: \
Price prediciton (model.py)

## Telegram bot (bot.py)

#### Commands
<li> /add </li>
Almost done. Need to add sync with database
<li> /delete </li>
Also sync with databse 
<li> /list </li>
Databse required
<li> /predict </li>
<li> /set_percent </li>

In [44]:
import telebot

token = '1686246223:AAFpGIs77TFZ89QErNnq65yTqzN7ULORvNQ'

bot = telebot.TeleBot(token)

@bot.message_handler(commands=['start'])
def greeting_command(message):
    text = f'''Hello {message.from_user.username}!
I\'m your stocks assistant bot.
I can help you to manage your stocks portfolio.

Here\'s something of what I can do.
Show you realtime prices of your stocksW
Predict the price of your stocks for a week
Notify you when you might want to sell stocks

For more help and commands /help

Created by @finlandcowboy'''
    
    bot.send_message(message.chat.id, text)

@bot.message_handler(commands=['help'])
def help_command(message):
    text = '''**Commands**
/add - Add stock
/delete - Delete Stock (if you sell it or smth)
/list - List your stocks
/predict - Predict your stock value for desirable amount of time
/set_percent - Set percent for notifying you about growth
    '''
    bot.send_message(message.chat.id, text, parse_mode='Markdown')

    
adding_stock_args = []
@bot.message_handler(commands=['add'])
def command_add(message):
    text = 'Enter a **ticker** (not a company name)'
    msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(msg, adding_ticker, parse_mode='Markdown')
    
def adding_ticker(message):
    adding_stock_args.append(message.text)
    text = 'Enter a price in **USD $$**'
    msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(msg, adding_price, parse_mode='Markdown')
    
def adding_price(message):
    adding_stock_args.append(int(message.text))
    text = 'Enter **amount** of stock you bought'
    msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(msg, adding_amount, parse_mode='Markdown')

    
def adding_amount(message):
    adding_stock_args.append(int(message.text))
    print(adding_stock_args)
    text = f'Alright, added a {adding_stock_args[0]} in amount of {adding_stock_args[2]} and sum price: {round(float(adding_stock_args[2]) * float(adding_stock_args[1]),2)}'
    bot.send_message(message.chat.id, text)
    
    '''
    ADD A STOCK INFO TO DATABASE HERE
    '''
delete_stock_args = []    
@bot.message_handler(commands=['delete'])
def command_delete(message):
    
    '''
    Add ticker try catch if ticker is not in list 
    
    
    except:
        print list of having stocks to suggest which to sell
    '''
    
    
    text = "What's ticker you want to sell?"
    msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(msg, delete_ticker)
    
def delete_ticker(message):
    delete_stock_args.append(message.text)
    amount = 0
    '''
    Query to get amount of stocks by ticker form database
    '''
    text = 'How many stocks you want to sell?\nRight now you have ' + str(amount)
    msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(msg, delete_amount)
    
def delete_amount(message):
    '''
    Add try catch if input isnt int 
    '''
    
    delete_stock_args.append(int(message.text))
    
    bot.send_message(message.chat.id, f'Okay, I sold {delete_stock_args[0]}\nIn amount of {delete_stock_args[1]}')



@bot.message_handler(commands=['list'])
def command_list(message):
    stocks_list = []
    '''
    Get list of tickers with current prices by API and database
    '''
    
    text = '**Your stocks list**'
    for stock in stocks_list:
        price = stocks.get_current_price(stock)
        text += f'\n{stock} {price}' 
    
    bot.send_message(message.chat.id, text, parse_mode='Markdown')
    
@bot.message_handler(commands=['set_percent'])
def command_set_percent(message):
    '''
    Get users percent to notify him
    '''
    text = "From what's percent of stock growth you want to be notified? Enter a number."
    msg = bot.send_message(message.chat.id, text)
    
    bot.register_next_step_handler(msg, set_percent_percent)

def set_percent_percent(message):
    '''
    Query to nisert percent to user_id (table user_percent)
    '''
    text = f'Your {message.text}% percent notifications is set'
    bot.send_message(message.chat.id, text)
    
@bot.message_handler(commands=['chatid'])
def command_chatid(message):
    bot.send_message(message.chat.id, message.chat.id)
    
plot_args = []
@bot.message_handler(commands=['plot'])
def command_plot(message):
    text = 'Enter a ticker for a plot'
    msg = bot.send_message(message.chat.id, message.chat.id)
    bot.register_next_step_handler(msg, plot_adding_ticker)
    
def plot_adding_ticker(message):
    text = 'Enter an interval for a plot.\nE.g. 1 week, 7 day, 1 year, 1 month'
    plot_args.append(message.text)
    msg = bot.send_message(message.chat.id, text)
    bot.register_next_step_handler(msg, plot_adding_interval)
    
def plot_adding_interval(message):
    plot_args.append(message.text)
    bot.send_chat_action(message.chat.id, 'upload_photo')
    
    bot.send_message(message.chat.id,f'Plot ticker: {plot_args[0]} with interval {plot_args[1]}')
    plot = plot_stock(plot_args[0], plot_args[1])
    bot.send_photo(message.chat.id, plot)
    
from threading import Thread
import schedule, time
    
bot.polling(True)    

## Stocks.py 

In [63]:
import uuid

In [235]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mpl_dates
from mpl_finance import candlestick_ochl
import mpl_finance as mpf
def get_current_price(ticker) -> float:
    tickerData = yf.Ticker(ticker)
    return np.round(tickerData.history(history='7d', history_input='2m').iloc[0].Close, 2)

def found_ticker(ticker):
    try: 
        yf.Ticker(ticker)
        return True
    except:
        return False
    

import finplot as fplt 
    
def save():
    fplt.screenshot(open(filename, 'wb'))
    
from io import BytesIO
def plot_stock(history_input, ticker):
    tickerData = yf.Ticker(ticker)
    history_inputs = ['week', 'day', 'month', 'year']
    for inter in history_inputs:
        if inter in history_input:
            history_input_args = history_input.split()
            history = ''
            for arg in history_input_args:
                try:
                    int(arg)
                    history += arg
                except:
                    history += arg[0]
            plt.style.use('ggplot')
            tickerDf = tickerData.history(period=history, interval='5m')
            tickerDf = tickerDf.rename(columns={'index':'Datetime', 'Date':'Datetime'})
            import uuid
            filename = uuid.uuid1().hex + '.png'
            print(f'Filename: {filename}')
            '''
                FIX SMTH HERE 
                IDK HOW IT WORKS 
                BUT I NEED TO LET THIS WORK SOMEHOW
            
            '''
            
            
            fplt.candlestick_ochl(tickerDf[['Open', 'Close', 'High', 'Low']])
            fplt.timer_callback(save, seconds=0.5, single_shot=True)
            fplt.show()
            fplt.close()
            import os
            if filename in os.listdir():
                print('File saved')
                return filename
            else:
                print('Error: File not saved')
            

In [234]:
plot = plot_stock('10 day', 'FB')

Filename: a9d37c68c9fc11eb92ab3024a99360be.png
Error: File not saved


In [178]:
plot

'632d5b65c9fa11ebaff43024a99360be.png'

In [211]:
history = '7d'
ticker = 'FB'

In [212]:
tickerData = yf.Ticker(ticker)
plt.style.use('ggplot')
tickerDf = tickerData.history(period=history, interval='1h')
tickerDf = tickerDf.rename(columns={'index':'Datetime', 'Date':'Datetime'})
import uuid


In [213]:
filename = uuid.uuid1().hex + '.png'
fplt.candlestick_ochl(tickerDf[['Open', 'Close', 'High', 'Low']])


In [223]:
fplt.show()
fplt.close()

In [220]:
def save():
    fplt.screenshot(open(filename, 'wb'))

In [221]:
fplt.timer_callback(save, seconds=0.5, single_shot=True)

In [217]:
filename

'5228763ac9fb11ebb37a3024a99360be.png'

## Database.py 

*Add stocks* \
*Delete stocks* \
*List stocks with current price & price for last 1 week, 1 day, 1 month* \
*Predict stocks price for 1 week*